In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import *
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
import mglearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
import os
import glob
from sklearn.svm import SVC
import tqdm



files = glob.glob("./*.csv")

# 説明変数となる行列X, 被説明変数となるy2を作成
base = 100 
day_ago = 75
pre_day=1
reset =True

#出来高やボリンジャーバンドなど指標の数
num_sihyou = 16

# すべてのCSVファイルから得微量作成
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4].astype(np.float) / temp2[base-1:len(temp2)-1, 4].astype(np.float)
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
     # 一目均衡表を追加 (9,26, 52) 
    para1 =9
    para2 = 26
    para3 = 52
    temp2_2 = np.c_[temp2, np.zeros((len(temp2), 3))]
    
    # 転換線 = （過去(para1)日間の高値 + 安値） ÷ 2
    for i in range(para1, len(temp2)):
        tmp_high =temp2[i-para1+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para1+1:i+1,3].astype(np.float)
        temp2_2[i, 9] = (np.max(tmp_high) + np.min(tmp_low))/2
        
    temp3[0:len(temp3), 8] = temp2_2[base:len(temp2), 9]/temp2[base:len(temp2), 4]

    # 基準線 = （過去(para2)日間の高値 + 安値） ÷ 2
    for i in range(para2, len(temp2)):
        tmp_high =temp2[i-para2+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para2+1:i+1,3].astype(np.float)
        temp2_2[i, 10] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 9] = temp2_2[base:len(temp2), 10]/temp2[base:len(temp2), 4]
        
    # 先行スパン1 = ｛ （転換値+基準値） ÷ 2 ｝を(para2)日先にずらしたもの
    temp3[0:len(temp3), 10] = (temp2_2[base-para2:len(temp2)-para2, 9] + temp2_2[base-para2:len(temp2)-para2, 10]) /2 /temp2[base:len(temp2), 4]

    # 先行スパン2 = ｛ （過去(para3)日間の高値+安値） ÷ 2 ｝を(para2)日先にずらしたもの
    for i in range(para3, len(temp2)):
        tmp_high =temp2[i-para3+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para3+1:i+1,3].astype(np.float)
        temp2_2[i, 11] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 11] = temp2_2[base-para2:len(temp2)-para2, 11]/temp2[base:len(temp2), 4]

    # 25日ボリンジャーバンド（±1, 2シグマ）を追加
    parab = 25
    for i in range(base, len(temp2)):
        tmp25 = temp2[i-parab+1:i+1,4].astype(np.float)
        temp3[i-base,12] = np.mean(tmp25) + 1.0* np.std(tmp25) 
        temp3[i-base,13] = np.mean(tmp25) - 1.0* np.std(tmp25) 
        temp3[i-base,14] = np.mean(tmp25) + 2.0* np.std(tmp25) 
        temp3[i-base,15] = np.mean(tmp25) - 2.0* np.std(tmp25)
     
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
             
    # Xに追加
    # X : すべての企業のデータ
    if reset:
        X = tempX[day_ago:]
        reset = False
    else:
        X = np.concatenate((X, tempX[day_ago:]), axis=0)
        
up_rate =1.0
# y : pre_day後の終値/当日終値
y = np.zeros(len(X))
y[0:len(y)-pre_day] = X[pre_day:len(X),0]


#それぞれの範囲でデータ数をカウント
a=b=0
yc = np.zeros(len(y))
for i in range(0, len(yc)):
    if y[i]>up_rate:
        yc[i]=1
        a+=1
    else :
        yc[i] = 0
        b+=1
print("0",a)
print("1",b)
print(y[len(yc)-2])
X_train, X_test, y_train, y_test = train_test_split(X, yc, random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=200000, alpha=0.001))])
param_grid = {'classifier__hidden_layer_sizes': [(10,), (100,), (500,)]}

grid = GridSearchCV(pipe, param_grid=param_grid, n_jobs=1, cv=2 ,return_train_score=False, scoring="f1")
grid.fit(X_train, y_train)

print(grid.cv_results_['mean_test_score'])
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

conf = confusion_matrix(y_test, grid.predict(X_test))
print(conf)

TypeError: 'module' object is not callable